In [65]:
from haystack.document_stores import ElasticsearchDocumentStore
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor
from haystack.nodes import BM25Retriever
from haystack.nodes import FARMReader
from haystack.utils import launch_es
from haystack import Pipeline
import logging
import os

In [66]:
#Inicializacion del modulo de logging para ver lo que sucede dentro de Haystack
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [67]:
# Conexión a nuestro almacen de documentos local
# Los documentos en formato txt se almacenarán aqui
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

In [68]:
# Creacion del Pipeline, Conversor de texto y Preprocesador
# El Pipeline se va a encargar de convertir nuestros archivos en documentos, hacer el split y guardarlos en el document store (elastic)
# Es como un pipeline de Jenkins
# El Conversor se encarga de convertir los datos dentro del TXT a un Document
# El PreProcesador se encarga de limpiar el formato, dividirlo en lineas y generar un formato entendible para el modelo a utilizar
indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)

In [69]:
# Se agregan los pasos del pipeline
# File --> TextConverter --> Preprocessor --> Document Store
indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])

In [ ]:
# Cargamos los datasets
# Este proceso solo se hace en la sincronizacion inicial o cuando se desee añadir o actualizar un documento
doc_dir = "gi_datasets"
files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline.run_batch(file_paths=files_to_index)


In [70]:
# Creamos el Retriever para los documentos almacenados
retriever = BM25Retriever(document_store=document_store)

In [ ]:
#mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es
#deepset/roberta-base-squad2
#model_to_use = "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
# Cargamos el modelo a utilizar
# 
model_to_use = "hy_fine_trained"
# TODO Validar la GPU
reader = FARMReader(model_name_or_path=model_to_use, use_gpu=True)

In [ ]:
# Creamos un nuevo pipeline para el proceso de Preguntas y Respuestas
# Dado una pregunta, buscara en la base de datos un contexto donde puede contener la respuesta, ayudandose del modelo que hemos indicado
# Entre mejor sea el modelo, mas precisa sera la respuesta y mas ambigua puede ser la pregunta
# Indagar en lo de GENERALIZACION
querying_pipeline = Pipeline()
# Query --> Retriever --> Reader 
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])

In [ ]:
# Realizamos una pregunta a nuestro pipeline
# Los parametros son "Cuantos documentos del retriever (elastic) quieres que tome como contexto"
# Y "Cuantos documentos del Reader (salida) quieres dar como resultado"
# Por asi decirlo, limitar en cuantos archivos voy a buscar y cuantas respuestas ofrezco al usuario
prediction = querying_pipeline.run(
    query="¿Qué estudia Laila?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

In [ ]:
# Buscar la clase Answer de Haystack
for aw in prediction["answers"]:
    print("Type: {} - Score: {}".format(aw.type, aw.score))
    print(aw.answer)

In [ ]:
#### GENERATIVE QA System (Plus Elastic Search & Custom Model QA)
# Esto es agregado a lo de arriba

In [71]:
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser

In [83]:
retriever = BM25Retriever(document_store=document_store, top_k=2)

In [84]:
rag_prompt = PromptTemplate(
    prompt="""Synthesize a comprehensive answer from the following text for the given question.
                             Provide a clear and concise response that summarizes the key points and information presented in the text.
                             Your answer should be in your own words.
                             \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
    output_parser=AnswerParser(),
)

In [85]:
prompt_node = PromptNode(model_name_or_path="google/flan-t5-small", default_prompt_template=rag_prompt)

INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


In [86]:
pipe = Pipeline()
pipe.add_node(component=retriever, name="retriever", inputs=["Query"])
pipe.add_node(component=prompt_node, name="prompt_node", inputs=["retriever"])

In [89]:
#documents = retriever.
output = pipe.run(query="¿Quien es la Shogun Raiden?")

Token indices sequence length is longer than the specified maximum sequence length for this model (982 > 512). Running this sequence through the model will result in indexing errors
WARNING - haystack.nodes.prompt.invocation_layer.hugging_face -  The prompt has been truncated from 982 tokens to 412 tokens so that the prompt length and answer length (100 tokens) fit within the max token limit (512 tokens). Shorten the prompt to prevent it from being cut off


In [90]:
print(output["answers"][0].answer)

ltima vez, ltima vez, ltima vez, ltima vez, ltima vez, ltima vez, ltima vez, ltima vez, ltima vez, ltima vez, ltima vez, ltima vez, l
